<a href="https://colab.research.google.com/github/VidunderGunder/ml-ady-vision/blob/master/vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLAdy Vision (Trash Detection)

In [3]:
%cd "/content"
!git clone https://github.com/VidunderGunder/ml-ady-vision.git
!git clone https://github.com/pedropro/TACO.git
!git clone https://github.com/ultralytics/yolov5
%cd "/content/yolov5"
%pip install -qr requirements.txt  # install dependencies
%cd "/content/TACO"
%pip install -qr requirements.txt  # install dependencies
%pip install split-folders
%cd "/content"

import torch
from IPython.display import Image, clear_output  # to display images
import os
import shutil
import numpy as np
import tqdm
from pycocotools.coco import COCO
import splitfolders
import string
import random

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download dataset

It's >2GB, so this will take some time.

Grab a coffee, some snacks or take a dump.

Feel free to read up on [YOLO](https://github.com/ultralytics/yolov5) or [TACO](https://github.com/pedropro/TACO) while you wait.

In [15]:
def safe_dir_copy(oldfolder, newfolder):
    tmpfolder = newfolder
    while True:
        if os.path.isdir(tmpfolder):
            tmpfolder = "-".join(
                (
                    newfolder, 
                    "".join(
                        random.choices(
                            string.ascii_uppercase + string.digits,
                            k=4,
                        )
                    ),
                )
            )
        else:
            break
    shutil.copytree(oldfolder, tmpfolder)

# data_in_root = os.path.exists("/content/data")
data_in_drive = os.path.exists("/content/drive/MyDrive/ml-ady-data/taco")
data_in_taco = os.path.exists("/content/TACO/data/batch_1")

if data_in_drive or data_in_taco:
    print("Data exists, skipping download")
else:
    %cd TACO
    !python download.py
    %cd "/content"

if data_in_taco and not data_in_drive:
    print("Copy data from taco to drive")
    safe_dir_copy(
        "/content/TACO/data/",
        "/content/drive/MyDrive/ml-ady-data/taco",
    )

# if data_in_drive and not data_in_root:
#     print("Copy data from drive to local")
#     safe_dir_copy(
#         "/content/drive/MyDrive/ml-ady-data/taco",
#         "/content/data/taco",
#     )

/content/TACO
Note. If for any reason the connection is broken. Just call me again and I will start where I left.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
^C
/content


## Convert TACO to YOLO

In [ ]:
%mv /content/ml-ady-vision/taco2yolo.py /content/

In [ ]:
!python taco2yolo.py

## Train

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/train

In [ ]:
%cd "/content/yolov5"
# !python train.py --img-size 640 --batch 16 --data yolov5s.yaml --weights yolov5s.pt --cache
python train.py --data taco.yaml --img-size 640 --cfg yolov5s.yaml --weights yolov5s.pt --device 0 --batch-size 16

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
Convert annotations from JSON to TXT: 515it [00:03, 169.64it/s]
Traceback (most recent call last):
  File "taco2yolo.py", line 115, in <module>
    os.path.join(save_image_path, save_name),
  File "/usr/lib/python3.6/shutil.py", line 245, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/usr/lib/python3.6/shutil.py", line 120, in copyfile
    with open(src, 'rb') as fsrc:
FileNotFoundError: [Errno 2] No such file or directory: './TACO/data/batch_14/000024.jpg'


### Training arguments

- `--weights` - initial weights path
- `--cfg` - model.yaml path
- `--data` - data.yaml path
- `--hyp` - hyperparameters path
- `--epochs` - training epochs (rounds/episodes)
- `--batch-size` - total batch size for all GPUs
- `--img-size` - \[train, test\] image sizes
- `--rect` - rectangular training
- `--resume` - resume most recent training
- `--nosave` - only save final checkpoint
- `--notest` - only test final epoch
- `--noautoanchor` - disable autoanchor check
- `--evolve` - evolve hyperparameters
- `--bucket` - gsutil bucket
- `--cache-images` - cache images for faster training
- `--image-weights` - use weighted image selection for training
- `--device` - cuda device, i.e. 0 or 0,1,2,3 or cpu
- `--multi-scale` - vary img-size +/- 50%%
- `--single-cls` - train multi-class data as single-class
- `--adam` - use torch.optim.Adam() optimizer
- `--sync-bn` - use SyncBatchNorm, only available in DDP mode
- `--local_rank` - DDP parameter, do not modify
- `--log-imgs` - number of images for W&B logging, max 100
- `--log-artifacts` - log artifacts, i.e. final trained model
- `--workers` - maximum number of dataloader workers
- `--project` - save to project/name
- `--name` - save to project/name
- `--exist-ok` - existing project/name ok, do not increment
- `--quad` - quad dataloader